In [24]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import fiona
import geopandas as gpd
import shapely

## Setup PostgreSQL/PostGIS Database

### Create Postgres database

In [4]:
conn = psycopg2.connect(
  dbname="postgres",
  user="postgres",
  password="",
  host="localhost",
  port="5432"
)
conn.autocommit = True 

In [5]:
cur = conn.cursor()

# Create a new database
db_name = "geocourse_data_engineering"
cur.execute(f"CREATE DATABASE {db_name};")

print(f"Database '{db_name}' created successfully.")

# Close the connection
cur.close()
conn.close()

Database 'geocourse_data_engineering' created successfully.


### Create Postgis Extension

In [18]:
conn = psycopg2.connect(
  dbname=db_name,
  user="postgres",
  password="",
  host="localhost",
  port="5432"
)
conn.autocommit = True 

In [ ]:
cur = conn.cursor()

# Create PostGIS extension
cur.execute(f"CREATE EXTENSION postgis;")
print("PostGIS extension successfully installed")

# Close the connection
cur.close()
conn.close()

PostGIS extension successfully installed


## Save data into PostGIS

In [8]:
engine = create_engine("postgresql://postgres:@localhost:5432/geocourse_data_engineering")

In [9]:
file_path = "../../transform/output.gpkg"

In [54]:
layers = fiona.listlayers(file_path)

layers

['airports_indonesia',
 'brn_protected_areas',
 'sgp_protected_areas',
 'tls_protected_areas',
 'twitter_word_counts',
 'candi_jawatengah',
 'admin_kawasan_jateng',
 'rdtr_takengon',
 'rdtr_takengon_vertices',
 'indonesian_cities',
 'twitter_users',
 'twitter_context_annotations']

In [11]:
airport_gdf = gpd.read_file(file_path, layer="airports_indonesia")

airport_gdf.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,icao_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated,source,geometry
0,26751,WADD,large_airport,Denpasar I Gusti Ngurah Rai International Airport,-8.748409,115.167123,14.0,AS,Indonesia,ID,...,WADD,DPS,None,http://www.angkasapura1.co.id/eng/location/bal...,https://en.wikipedia.org/wiki/Ngurah_Rai_Inter...,"WRRR, Bali, Denpasar International Airport, De...",1200,2025-02-21T20:46:40+00:00,OurAirports,POINT Z (115.16712 -8.74841 14)
1,26835,WIII,large_airport,Soekarno-Hatta International Airport,-6.125570,106.655998,34.0,AS,Indonesia,ID,...,WIII,CGK,None,http://www.jakartasoekarnohattaairport.com/,https://en.wikipedia.org/wiki/Soekarno-Hatta_I...,"JKT, Cengkareng, Java",51600,2013-02-07T12:57:41+00:00,OurAirports,POINT Z (106.656 -6.12557 34)
2,26787,WARJ,medium_airport,Adisutjipto International Airport,-7.788180,110.431999,379.0,AS,Indonesia,ID,...,None,JOG,None,http://adisutjipto-airport.co.id/,https://en.wikipedia.org/wiki/Adisutjipto_Inte...,"WIIJ, WARJ, Adisucupto",500,2022-10-01T23:14:17+00:00,OurAirports,POINT Z (110.432 -7.78818 379)
3,26789,WARR,large_airport,Juanda International Airport,-7.379830,112.787003,9.0,AS,Indonesia,ID,...,WARR,SUB,None,http://www.juanda-airport.com/,https://en.wikipedia.org/wiki/Juanda_Internati...,WRSJ,51150,2022-10-01T23:14:37+00:00,OurAirports,POINT Z (112.787 -7.37983 9)
4,299629,WADL,medium_airport,Lombok International Airport,-8.759962,116.278169,319.0,AS,Indonesia,ID,...,WADL,LOP,None,http://www.lombok-airport.co.id/,https://en.wikipedia.org/wiki/Lombok_Internati...,"Zainuddin Abdul Madjid, Praya",500,2025-02-21T23:53:28+00:00,OurAirports,POINT Z (116.27817 -8.75996 319)


In [12]:
airport_gdf.to_postgis("airport_indonesia", engine, if_exists="replace", index=False)

print("Airport data successfully uploaded into PostgreSQL/PostGIS")

InvalidParameterValue: Column has Z dimension but geometry does not
CONTEXT:  COPY airport_indonesia, line 197, column geometry: "0101000020E6100000BF9A0304733E5D401F4AB4E4F134F3BF"


In [20]:
def to_2d(geom):
  if geom.has_z:
    return shapely.wkb.loads(shapely.wkb.dumps(geom, output_dimension=2))
  return geom 

airport_gdf.geometry = airport_gdf.geometry.apply(to_2d)

In [41]:
airport_gdf.to_postgis("airport_indonesia", engine, if_exists="replace", index=False)

print("Airport data successfully uploaded into PostgreSQL/PostGIS")

Airport data successfully uploaded into PostgreSQL/PostGIS


### Check if data is saved into PostGIS

In [ ]:
query = "SELECT id, type, name, geometry FROM airport_indonesia"

df = gpd.read_postgis(query, engine, geom_col="geometry")

df.head()

,id,type,name,geometry
0,26751,large_airport,Denpasar I Gusti Ngurah Rai International Airport,POINT (115.16712 -8.74841)
1,26835,large_airport,Soekarno-Hatta International Airport,POINT (106.656 -6.12557)
2,26787,medium_airport,Adisutjipto International Airport,POINT (110.432 -7.78818)
3,26789,large_airport,Juanda International Airport,POINT (112.787 -7.37983)
4,299629,medium_airport,Lombok International Airport,POINT (116.27817 -8.75996)


In [34]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        638 non-null    int64   
 1   type      638 non-null    object  
 2   name      638 non-null    object  
 3   geometry  638 non-null    geometry
dtypes: geometry(1), int64(1), object(2)
memory usage: 20.1+ KB


### Upload all layers into PostGIS

In [64]:
geo_layers = []
nongeo_layers = []

for layer in layers[1:]:
  with fiona.open(file_path, layer=layer) as src:
    has_geometry = False      
    for feature in src:
      geometry = feature.get("geometry")
      if geometry:
        geom_type = geometry["type"]
        print(f"Layer '{layer}' contains valid geometry")
        has_geometry = True
        break
    
    if has_geometry:
      geo_layers.append(layer)
    else:
      nongeo_layers.append(layer)
      print(f"Layer '{layer}' has no valid geometries.")

Layer 'brn_protected_areas' contains valid geometry
Layer 'sgp_protected_areas' contains valid geometry
Layer 'tls_protected_areas' contains valid geometry
Layer 'twitter_word_counts' contains valid geometry
Layer 'candi_jawatengah' contains valid geometry
Layer 'admin_kawasan_jateng' contains valid geometry
Layer 'rdtr_takengon' contains valid geometry
Layer 'rdtr_takengon_vertices' contains valid geometry
Layer 'indonesian_cities' contains valid geometry
Layer 'twitter_users' has no valid geometries.
Layer 'twitter_context_annotations' has no valid geometries.


In [65]:
geo_layers

['brn_protected_areas',
 'sgp_protected_areas',
 'tls_protected_areas',
 'twitter_word_counts',
 'candi_jawatengah',
 'admin_kawasan_jateng',
 'rdtr_takengon',
 'rdtr_takengon_vertices',
 'indonesian_cities']

In [66]:
nongeo_layers

['twitter_users', 'twitter_context_annotations']

In [67]:
for layer in geo_layers:
  gdf = gpd.read_file(file_path, layer=layer)
  gdf.to_postgis(layer, engine, if_exists="replace", index=False)
  print(f"{layer} successfully uploaded into PostgreSQL/PostGIS")

brn_protected_areas successfully uploaded into PostgreSQL/PostGIS
sgp_protected_areas successfully uploaded into PostgreSQL/PostGIS
tls_protected_areas successfully uploaded into PostgreSQL/PostGIS
twitter_word_counts successfully uploaded into PostgreSQL/PostGIS
candi_jawatengah successfully uploaded into PostgreSQL/PostGIS
admin_kawasan_jateng successfully uploaded into PostgreSQL/PostGIS
rdtr_takengon successfully uploaded into PostgreSQL/PostGIS
rdtr_takengon_vertices successfully uploaded into PostgreSQL/PostGIS
indonesian_cities successfully uploaded into PostgreSQL/PostGIS


In [69]:
for layer in nongeo_layers:
  gdf = gpd.read_file(file_path, layer=layer)
  gdf.to_sql(layer, engine, if_exists="replace", index=False)
  print(f"{layer} successfully uploaded into PostgreSQL")

twitter_users successfully uploaded into PostgreSQL
twitter_context_annotations successfully uploaded into PostgreSQL
